In [30]:
from os.path import dirname
import os
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [31]:
pd.set_option('display.max_columns', 50)

In [82]:
%load_ext autoreload
%autoreload 2
import brandear_est as be

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
MIMIC_DIR = os.path.join(os.getcwd(), "../../data/mimic/")
INPUT_DIR = os.path.join(os.getcwd(), "../../data/input/")
IMD_DIR = os.path.join(os.getcwd(), "../../data/intermediate/")
SUBMIT_DIR = os.path.join(os.getcwd(), "../../data/submit/")

dset_to_period = {
    "train": {"oldest": datetime.datetime(2019, 9, 3, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 10, 0, 0, 0)},
    "valid_for_train": {"oldest": datetime.datetime(2019, 9, 10, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 17, 0, 0, 0),},    
    "valid_for_sub": {"oldest": datetime.datetime(2019, 9, 17, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 24, 0, 0, 0),},
    "submission": {"oldest": datetime.datetime(2019, 9, 24, 0, 0, 0),
              "newest": datetime.datetime(2019, 10, 1, 0, 0, 0),},    
}

In [86]:
try:
    auction= pd.read_pickle(IMD_DIR + "202001151119" + "_auction.pkl")
    watch= pd.read_pickle(IMD_DIR + "202001151119" + "_watch.pkl")
    bid= pd.read_pickle(IMD_DIR + "202001151119" + "_bid.pkl")
    bid_success= pd.read_pickle(IMD_DIR + "202001151119" + "_bid_success.pkl")
    sample_sub = pd.read_csv(INPUT_DIR + "sample_submission.csv")    
    
    sub_users = sample_sub[["KaiinID"]].drop_duplicates()
    
except:
    auction = pd.read_csv(INPUT_DIR + "auction.csv")
    watch = pd.read_csv(INPUT_DIR + "watchlist.csv")
    bid = pd.read_csv(INPUT_DIR + "shudounyuusatsu.csv")
    bid_success = pd.read_csv(INPUT_DIR + "rakusatsu.csv")
    sample_sub = pd.read_csv(INPUT_DIR + "sample_submission.csv")

    auction = to_datetime(auction)
    watch = to_datetime(watch)
    bid = to_datetime(bid)
    bid_success = to_datetime(bid_success)

    watch = watch.merge(auction, on="AuctionID")
    bid = bid.merge(auction, on="AuctionID")
    bid_success = bid_success.merge(auction, on="AuctionID")

    sub_users = sample_sub[["KaiinID"]].drop_duplicates()

    to_pickle(auction, IMD_DIR, "auction.pkl")
    to_pickle(watch, IMD_DIR, "watch.pkl")
    to_pickle(bid, IMD_DIR, "bid.pkl")
    to_pickle(bid_success, IMD_DIR, "bid_success.pkl")

In [5]:
%%time
try:
    train_dataset = pd.read_pickle(IMD_DIR + "202001152052" + "_train_dataset.pkl")
    valid_for_train_dataset = pd.read_pickle(IMD_DIR + "202001152052" + "valid_for_train_dataset.pkl")
    valid_for_sub_dataset = pd.read_pickle(IMD_DIR + "202001152055" + "valid_for_sub_dataset.pkl")
    sub_dataset = pd.read_pickle(IMD_DIR + "202001152057" + "_sub_dataset.pkl")    
except:
    train_dataset = be.build_dataset(watch, bid, auction, "train", dset_to_period)
    valid_for_train_dataset = be.build_dataset(watch, bid, auction, "valid_for_train", dset_to_period)
    valid_for_sub_dataset = be.build_dataset(watch, bid, auction, "valid_for_sub", dset_to_period)    
    sub_dataset = be.build_dataset(watch, bid, auction, "submission", dset_to_period, sub_users)
    
    be.to_pickle(train_dataset, IMD_DIR, "train_dataset.pkl")
    be.to_pickle(valid_for_train_dataset, IMD_DIR, "valid_for_train_dataset.pkl")
    be.to_pickle(valid_for_sub_dataset, IMD_DIR, "valid_for_sub_dataset.pkl")    
    be.to_pickle(sub_dataset, IMD_DIR, "sub_dataset.pkl")

CPU times: user 702 ms, sys: 2.21 s, total: 2.91 s
Wall time: 3.05 s


In [44]:
train_dataset = be.build_dataset(watch, bid, auction, "train", dset_to_period)

/Users/nobuyuki.ishida/Practice/competition/nishika/Brandear/code/notebook/brandear_est/feature_engineering.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df.loc[tmp_time_col] = feature_df[time_col].apply(lambda d: (oldest_dtime - d).days)


In [112]:
lgb_rank = be.LgbLambdaLank()

In [66]:
lgb_rank.train(train_data=train_dataset, valid_data=valid_for_train_dataset)

[1]	valid_0's ndcg@20: 0.733728
[2]	valid_0's ndcg@20: 0.734831
[3]	valid_0's ndcg@20: 0.73742
[4]	valid_0's ndcg@20: 0.737218
[5]	valid_0's ndcg@20: 0.737732
[6]	valid_0's ndcg@20: 0.741859
[7]	valid_0's ndcg@20: 0.742371
[8]	valid_0's ndcg@20: 0.74212
[9]	valid_0's ndcg@20: 0.742264
[10]	valid_0's ndcg@20: 0.742406
[11]	valid_0's ndcg@20: 0.742672
[12]	valid_0's ndcg@20: 0.742434
[13]	valid_0's ndcg@20: 0.742604
[14]	valid_0's ndcg@20: 0.742871
[15]	valid_0's ndcg@20: 0.742825
[16]	valid_0's ndcg@20: 0.742711
[17]	valid_0's ndcg@20: 0.742913
[18]	valid_0's ndcg@20: 0.7431
[19]	valid_0's ndcg@20: 0.743146
[20]	valid_0's ndcg@20: 0.742816
[21]	valid_0's ndcg@20: 0.742816
[22]	valid_0's ndcg@20: 0.742838
[23]	valid_0's ndcg@20: 0.743045
[24]	valid_0's ndcg@20: 0.743189
[25]	valid_0's ndcg@20: 0.743259
[26]	valid_0's ndcg@20: 0.743139
[27]	valid_0's ndcg@20: 0.74338
[28]	valid_0's ndcg@20: 0.743424
[29]	valid_0's ndcg@20: 0.743507
[30]	valid_0's ndcg@20: 0.743805
[31]	valid_0's ndcg@20: 

In [113]:
lgb_rank.retrain(train_data=valid_for_train_dataset)

In [153]:
pred = lgb_rank.predict(valid_for_sub_dataset)

TypeError: predict() got an unexpected keyword argument 'drop'

In [154]:
pred

,KaiinID,AuctionID,score
1345305,22,2193,-0.125316
1204420,22,82220,-0.988238
3150773,22,122841,0.377487
403824,22,139072,0.407981
2141495,22,147277,0.017018
...,...,...,...
2619566,277891,4089851,-0.783706
2619611,277891,4090315,-1.147872
2427160,277891,4097355,-1.409389
38148,277891,4104172,-1.409389


In [119]:
sub = be.adjust_sub_form(sub_users, pred, drop=True)

In [120]:
sub.head()

,KaiinID,AuctionID
1180001,24,3416773
849872,24,2459057
432319,24,1250994
1161045,24,3362272
157857,24,457253


In [121]:
valid_for_sub_dataset

,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,DanjobetsuID,watch_ua_cnt,watch_ua_newest,watch_ua_oldest,watch_period,watch_AuctionID_cnt,watch_ShouhinID_cnt,watch_BrandID_cnt,watch_LineID_cnt,watch_KaiinID_ShouhinID_cnt,watch_KaiinID_BrandID_cnt,watch_KaiinID_GenreGroupID_cnt,watch_KaiinID_LineID_cnt,bid_ua_cnt,bid_ua_newest,bid_ua_oldest,bid_period,bid_AuctionID_cnt,bid_ShouhinID_cnt,bid_BrandID_cnt,bid_LineID_cnt,bid_KaiinID_ShouhinID_cnt,bid_KaiinID_BrandID_cnt,bid_KaiinID_GenreGroupID_cnt,bid_KaiinID_LineID_cnt,watch_actioned,bid_actioned
0,134759,1862947,0,6,2,1.0,356.0,356.0,0.0,9.0,9.0,9757.0,3874863,1.0,4.0,101.0,479.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,821.0,560636.0,-1.0,-1.0,17.0,65.0,0.0,0.0
1,75217,1862947,0,6,2,1.0,380.0,380.0,0.0,9.0,9.0,9757.0,3874863,1.0,12.0,40.0,998.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,821.0,560636.0,-1.0,-1.0,-1.0,41.0,0.0,0.0
2,88688,1862947,0,6,2,1.0,367.0,367.0,0.0,9.0,9.0,9757.0,3874863,1.0,32.0,20.0,1074.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,821.0,560636.0,-1.0,9.0,3.0,85.0,0.0,0.0
3,151198,1862947,0,6,2,1.0,367.0,367.0,0.0,9.0,9.0,9757.0,3874863,1.0,113.0,855.0,4253.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,821.0,560636.0,-1.0,6.0,13.0,128.0,0.0,0.0
4,51157,1862947,0,6,2,-1.0,-1.0,-1.0,-1.0,9.0,9.0,9757.0,3874863,-1.0,114.0,1564.0,10634.0,1.0,355.0,355.0,0.0,1.0,1.0,821.0,560636.0,1.0,57.0,285.0,1370.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446149,95156,704429,0,5,0,1.0,281.0,281.0,0.0,2.0,2.0,240879.0,91278,1.0,10.0,11.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,44189.0,22690.0,-1.0,1.0,1.0,1.0,0.0,0.0
3446150,95156,3005528,0,5,0,1.0,281.0,281.0,0.0,4.0,4.0,240879.0,91278,1.0,10.0,11.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,44189.0,22690.0,-1.0,1.0,1.0,1.0,0.0,0.0
3446151,95156,42681,0,5,0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,240879.0,91278,1.0,10.0,11.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,44189.0,22690.0,-1.0,1.0,1.0,1.0,0.0,0.0
3446152,95156,216600,0,5,0,1.0,281.0,281.0,0.0,1.0,1.0,240879.0,91278,1.0,10.0,11.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,44189.0,22690.0,-1.0,1.0,1.0,1.0,0.0,0.0


In [146]:
valid_for_sub_dataset.query("(watch_actioned == 1) | (bid_actioned == 1)")[["KaiinID", "AuctionID", "watch_actioned", "bid_actioned"]]

,KaiinID,AuctionID,watch_actioned,bid_actioned
1088,155725,2314801,1.0,0.0
1468,79931,2534463,0.0,1.0
1487,37168,1607982,1.0,0.0
1762,10091,4049171,1.0,0.0
1854,55741,2164090,1.0,0.0
...,...,...,...,...
3445537,152332,1816854,0.0,1.0
3445901,137478,303138,0.0,1.0
3445917,137478,3421965,0.0,1.0
3446054,114235,3996335,0.0,1.0


In [143]:
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    return np.sum((2 ** r - 1) / np.log2(np.arange(2, r.size + 2)))

def ndcg_at_k(r, k):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

In [189]:
watch_true = valid_for_sub_dataset.query("(watch_actioned == 1)")[["KaiinID", "AuctionID"]]
watch_true["score"] = 1
bid_true = valid_for_sub_dataset.query("(bid_actioned == 1)")[["KaiinID", "AuctionID"]]
bid_true["score"] = 2
scored_data = (
    sub.merge(pd.concat([watch_true, bid_true]), on=["KaiinID", "AuctionID"], how="left").fillna(0)
    .sort_values(["KaiinID", "AuctionID", "score"], ascending=[True, True, False])
)

In [195]:
sum_ndcg = 0
for _, df in scored_data.groupby("KaiinID"):
    sum_ndcg += ndcg_at_k(df["score"].values.tolist(), 20)
ndcg = sum_ndcg / scored_data["KaiinID"].nunique()

In [196]:
ndcg

0.05897788489111075

In [156]:


    pd.concat([
        valid_for_sub_dataset.query("(watch_actioned == 1)")[["KaiinID", "AuctionID", "watch_actioned"]]
        .rename(columns={"watch_actioned": "flg"}), 
        valid_for_sub_dataset.query("(bid_actioned == 1)")[["KaiinID", "AuctionID", "bid_actioned"]]
        .rename(columns={"bid_actioned": "flg"})
    ])).astype(int).merge(sub, ["KaiinID", "AuctionID"], "right")

KeyError: 'right'

In [ ]:
y_true = 

In [108]:
def ndcg_score(y_true, y_pred, k=20):
    y_pred['rank'] = y_pred.groupby('KaiinID')['AuctionID'].cumcount()
    return y_pred

In [109]:
ndcg_score(sub,sub)

,KaiinID,AuctionID,rank
687443,24,1782294.0,0
1297636,24,3366893.0,1
1248869,24,3240222.0,2
1380594,24,3580719.0,3
566542,24,1467774.0,4
...,...,...,...
3553648,277891,1257339.0,15
3554398,277891,4059566.0,16
3554240,277891,3469952.0,17
3553698,277891,1472717.0,18


In [ ]:
sub_valid[["KaiinID", "AuctionID"]].to_csv(datetime.datetime.now().strftime("%Y%m%d%H%M") + "_submit.csv",
                                           index=False)

In [27]:
train_sampled_dataset = (
    pd.concat([train_dataset.query("(watch_actioned == 0) & (bid_actioned == 0)").sample(n=70000, random_state=10),
               train_dataset.query("(watch_actioned == 1) | (bid_actioned == 1)")])
)

In [28]:
train_sampled_dataset.sort_values("KaiinID", inplace=True)
train_label = np.array(train_sampled_dataset[["watch_actioned", "bid_actioned"]].astype(int)).max(axis=1)
train_weight = (
    np.stack([
        np.array(train_sampled_dataset["watch_actioned"].astype(int)),
        (np.array(train_sampled_dataset["bid_actioned"]).astype(int) * 2),
        np.ones((train_sampled_dataset.shape[0], ))
    ], 1).max(axis=1)
)
train_group = train_sampled_dataset[["KaiinID", "AuctionID"]].groupby("KaiinID", as_index=False).count().sort_values("KaiinID")["AuctionID"]

In [29]:
valid_dataset.sort_values("KaiinID", inplace=True)
valid_label = np.array(valid_dataset[["watch_actioned", "bid_actioned"]].astype(int)).max(axis=1)
valid_weight = (
    np.stack([np.array(valid_dataset["watch_actioned"].astype(int)),
              (np.array(valid_dataset["bid_actioned"]).astype(int) * 2),
            np.ones((valid_dataset.shape[0], ))             
    ], 1).max(axis=1)
)
valid_group = valid_dataset[["KaiinID", "AuctionID"]].groupby("KaiinID", as_index=False).count().sort_values("KaiinID")["AuctionID"]

In [30]:
lbg_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    "ndcg_at": 20,
    "nround": 500,
    "learning_rate": 0.01,
    "max_depth": 6,
    "num_leaves": 127,
}

drop_cols = ["KaiinID", "AuctionID", "watch_actioned", "bid_actioned"]

lgbtrain = lgb.Dataset(data=np.array(train_sampled_dataset.drop(drop_cols, axis=1)),
                      label=train_label,
                      weight=train_weight,
                      group=train_group
                      )
lgbvalid = lgb.Dataset(data=np.array(valid_dataset.drop(drop_cols, axis=1)),
                      label=valid_label,
                      weight=valid_weight,
                      group=valid_group)

lgb_clf = lgb.train(
    params=lbg_params,
    train_set=lgbtrain,
    valid_sets=lgbvalid
)

[1]	valid_0's ndcg@20: 0.727767
[2]	valid_0's ndcg@20: 0.738501
[3]	valid_0's ndcg@20: 0.740531
[4]	valid_0's ndcg@20: 0.740792
[5]	valid_0's ndcg@20: 0.741505
[6]	valid_0's ndcg@20: 0.741369
[7]	valid_0's ndcg@20: 0.742034
[8]	valid_0's ndcg@20: 0.741496
[9]	valid_0's ndcg@20: 0.74144
[10]	valid_0's ndcg@20: 0.741734
[11]	valid_0's ndcg@20: 0.741714
[12]	valid_0's ndcg@20: 0.743711
[13]	valid_0's ndcg@20: 0.744204
[14]	valid_0's ndcg@20: 0.744784
[15]	valid_0's ndcg@20: 0.74491
[16]	valid_0's ndcg@20: 0.744952
[17]	valid_0's ndcg@20: 0.744973
[18]	valid_0's ndcg@20: 0.744993
[19]	valid_0's ndcg@20: 0.745098
[20]	valid_0's ndcg@20: 0.745052
[21]	valid_0's ndcg@20: 0.74511
[22]	valid_0's ndcg@20: 0.745052
[23]	valid_0's ndcg@20: 0.745299
[24]	valid_0's ndcg@20: 0.745218
[25]	valid_0's ndcg@20: 0.744763
[26]	valid_0's ndcg@20: 0.744838
[27]	valid_0's ndcg@20: 0.745229
[28]	valid_0's ndcg@20: 0.745078
[29]	valid_0's ndcg@20: 0.744791
[30]	valid_0's ndcg@20: 0.744781
[31]	valid_0's ndcg@20

In [44]:
importance = pd.DataFrame(lgb_clf.feature_importance(), 
                          index=train_sampled_dataset.drop(drop_cols, axis=1).columns,
                          columns=['importance'])
importance

,importance
SaishuppinKaisuu,579
ConditionID,112
DanjobetsuID,151
watch_ua_cnt,126
watch_ua_newest,353
watch_ua_oldest,433
watch_period,9
watch_AuctionID_cnt,241
watch_ShouhinID_cnt,261
watch_BrandID_cnt,298


In [170]:
valid_sampled_dataset = (
    pd.concat([valid_dataset.query("(watch_actioned == 0) & (bid_actioned == 0)").sample(n=70000),
               valid_dataset.query("(watch_actioned == 1) | (bid_actioned == 1)")])
)

valid_sampled_dataset.sort_values("KaiinID", inplace=True)
valid_label = np.array(valid_sampled_dataset[["watch_actioned", "bid_actioned"]].astype(int)).max(axis=1)
valid_weight = (
    np.stack([np.array(valid_sampled_dataset["watch_actioned"].astype(int)),
              (np.array(valid_sampled_dataset["bid_actioned"]).astype(int) * 2),
            np.ones((valid_sampled_dataset.shape[0], ))             
    ], 1).max(axis=1)
)
valid_group = valid_sampled_dataset[["KaiinID", "AuctionID"]].groupby("KaiinID", as_index=False).count().sort_values("KaiinID")["AuctionID"]

In [171]:
lgbvalid = lgb.Dataset(data=np.array(valid_sampled_dataset.drop(drop_cols, axis=1)),
                      label=valid_label,
                      weight=valid_weight,
                      group=valid_group)

In [173]:
lgb_for_sub = lgb.train(
    params=lbg_params,
    train_set=lgbvalid,
)
pred = lgb_for_sub.predict(
    data=np.array(sub_dataset.sort_values(["KaiinID", "AuctionID"]).drop(["AuctionID", "KaiinID"], axis=1)),
    group=np.array(sub_dataset[["KaiinID", "AuctionID"]].groupby("KaiinID", as_index=False).count().sort_values("KaiinID")["AuctionID"])
)
pd.Series(pred).unique()

array([-2.22717918, -1.91036766, -0.38131559, ...,  1.13413547,
        1.74235853,  0.86493182])

In [165]:
# lbg_params = {
#     "objective": "binary",
#     "nround": 500,
#     "learning_rate": 0.01,
#     "max_depth": 6,
#     "num_leaves": 127
# }

# drop_cols = ["KaiinID", "AuctionID", "watch_actioned", "bid_actioned"]

# watch_model = lgb.train(
#     params=lbg_params,
#     train_set=lgb.Dataset(np.array(train_dataset.drop(drop_cols, axis=1)),label = np.array(train_dataset["watch_actioned"])),
#     valid_sets=lgb.Dataset(np.array(valid_dataset.drop(drop_cols, axis=1)),label = np.array(valid_dataset["watch_actioned"])),    
# )
# nyuusatsu_model = lgb.train(
#     params=lbg_params,
#     train_set=lgb.Dataset(np.array(train_dataset.drop(drop_cols, axis=1)),label = np.array(train_dataset["bid_actioned"])),
#     valid_sets=lgb.Dataset(np.array(valid_dataset.drop(drop_cols, axis=1)),label = np.array(valid_dataset["bid_actioned"])),    
# )    
# watch_pred = watch_model.predict(np.array(sub_dataset.drop(["AuctionID", "KaiinID"], axis=1)))
# nyuusatsu_pred = nyuusatsu_model.predict(np.array(sub_dataset.drop(["AuctionID", "KaiinID"], axis=1)))

# sub_dataset["watch_pred"] = watch_pred
# sub_dataset["nyuusatsu_pred"] = nyuusatsu_pred
# sub_pred = sub_dataset[["KaiinID", "AuctionID", "watch_pred", "nyuusatsu_pred"]]
# sub_pred["score"] = sub_pred["watch_pred"] * 0.2 + sub_pred["nyuusatsu_pred"] * 0.8

In [174]:
sub_pred = sub_dataset[["KaiinID", "AuctionID"]]
sub_pred["score"] = pred
sub_ranks = sub_users.merge(sub_pred, on="KaiinID", how="left")[["KaiinID", "AuctionID", "score"]]

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [176]:
sub_ranks = comple_submit_auc(sub_ranks)
sub_ranks.sort_values(['KaiinID', 'score'], ascending=[True, False], inplace=True)
sub_ranks['rank'] = sub_ranks.groupby('KaiinID')['score'].cumcount()
sub_valid = sub_ranks.query("rank < =19")
sub_valid.sort_values(['KaiinID', 'score'], ascending=[True, False], inplace=True)
sub_valid[["KaiinID", "AuctionID"]].to_csv(datetime.datetime.now().strftime("%Y%m%d%H%M") + "_submit.csv", index=False)

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
